# GVC participation rate

In [42]:
import numpy as np
import pandas as pd
import duckdb

## By export sectors

In [47]:
file = 'ta72-os.parquet'
outputfile = 'gvcp.csv'

In [49]:
df = duckdb.sql(
    f"""
    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{file}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{file}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{file}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

In [45]:
df['GVCf'] = df['REX1'] + df['REX2'] + df['REX3'] + df['REF1'] + df['REF2']
df['GVCb'] = df['FVA'] + df['PDC1'] + df['PDC2']
df['GVC'] = df['GVCf'] + df['GVCb']

df['GVCP_trade_f'] = df['GVCf'] / df['Exports']
df['GVCP_trade_b'] = df['GVCb'] / df['Exports']
df['GVCP_trade'] = df['GVC'] / df['Exports']

df = df[['t', 's', 'agg', 'i', 'Exports', 'GVCf', 'GVCb', 'GVC', 'GVCP_trade_f', 'GVCP_trade_b', 'GVCP_trade']]

In [46]:
df.to_csv(f'../data/final/{outputfile}', index=False)